<a href="https://colab.research.google.com/github/ankit-kothari/spacy_playground/blob/master/SPACY_PLAYGROUND.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##GENERAL
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from collections import Counter
import csv
import re

##SPACY
import spacy
nlp = spacy.load('en_core_web_sm')
#nlp = spacy.load('en_core_web_md')
#nlp = spacy.load('en_core_web_lg')
from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy.pipeline import SentenceSegmenter
from spacy import displacy

##NLTK
import nltk
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer

##SKLEARN
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.decomposition import NMF



# Data Preprocessing and Exploration

###Documentation

In [ ]:
###Annotations: https://spacy.io/api/annotation
### Visualization: https://spacy.io/usage/visualizers
### Named EntitY Recog https://spacy.io/usage/linguistic-features#named-entities
#### https://spacy.io/api/annotation#dependency-parsing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


###Loading the Data

In [ ]:
data = pd.read_csv('/content/drive/My Drive/data/Womens Clothing E-Commerce Reviews 2.csv')
data.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


1. Identifying the maximum number of reviews by Class Name 
2. Identifying the Clothing ID whihc has the maximum Reviews for the 

###Data Exploration

In [ ]:
data['Class Name'].value_counts()[0:5]

Dresses     6319
Knits       4843
Blouses     3097
Sweaters    1428
Pants       1388
Name: Class Name, dtype: int64

In [ ]:
data_dress = data[(data['Class Name']=='Dresses')]
data_dress['Clothing ID'].value_counts()[0:5]

1078    1024
1094     756
1081     582
1110     480
1095     327
Name: Clothing ID, dtype: int64

1. Filtering the data with the Class Name which has the maximum reviews **DRESSS**
2. Filtering the data with Clothing ID **1078** which has the maximum reviews for Class Name **DRESS**

###Filtering the data 

In [ ]:
data_dress = data[(data['Class Name']=='Dresses') & (data['Clothing ID']==1078)]

In [ ]:
data_dress.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
69,69,1078,56,Great summer fabric!,"I really wanted this to work. alas, it had a s...",3,0,1,General Petite,Dresses,Dresses
90,90,1078,51,Sweet flattering dress,"I love cute summer dresses and this one, espec...",4,1,0,General Petite,Dresses,Dresses
117,117,1078,32,NaN,This is the perfect summer dress. it can be dr...,5,1,2,General Petite,Dresses,Dresses
467,467,1078,61,Great sweater dress!,"Nice fit and flare style, not clingy at all. i...",5,1,1,General,Dresses,Dresses
470,470,1078,33,"Cute, but cheap",When i first opened this dress and tried it on...,3,0,0,General,Dresses,Dresses


###ANALYZING THE REVIEWS FOR **DRESSES** AND CLOTHING ID **1078**

1. Converting Reviews into Spacy Object

In [ ]:
#for i in data_dress['Review Text']:
    #print(i)   
##print(".".join(data_dress['Review Text']))
dress_review=data_dress['Review Text'].str.cat(sep='\n')
doc = nlp(dress_review)
doc[1200:1600]

skirt.

love it! will be easy to wear casually and work appropriate, too. the sale price was a huge bonus.
I love this dress because its very playful and bouncy. it puts me in a light hearted mood when i wear it. i originally wanted to buy the grey color but my store only had the navy, so i tried it on. the navy is brighter and more colorful than it looks on line and the stripes are more varied in color than in the picture - so its quite appealing and vibrant. the lines of the dress are also quite flattering. all in all, its a fun dress!
This dress is comfortable as well as flattering, which does not happen very often!
looks good with navy tights too!
Based on some reviews i decided to get the regular xs, even tho i am an xs petite (5'2, 107 lb, 32c) i found the fit to be flattering -- fitted enough but not too loose or tight. the length is perfect and work appropriate, and the material has some weight to it. the grey one is colorful and cute. the only problem is the bagginess of the a

2. UNDERSTANDING THE SENTENCE STRUCTURE

In [ ]:
for token in doc[0:10]:
    print(f'{token.text:{10}} {token.lemma_:{10}} {token.pos_:{6}} {token.dep_:{12}} {spacy.explain(token.tag_)}')

I          -PRON-     PRON   nsubj        pronoun, personal
really     really     ADV    advmod       adverb
wanted     want       VERB   ROOT         verb, past tense
this       this       DET    nsubj        determiner
to         to         PART   aux          infinitival "to"
work       work       VERB   ccomp        verb, base form
.          .          PUNCT  punct        punctuation mark, sentence closer
alas       alas       INTJ   intj         interjection
,          ,          PUNCT  punct        punctuation mark, comma
it         -PRON-     PRON   nsubj        pronoun, personal


3. CONTENTS OF A NLP PIPELINE IN SPACY

In [ ]:
nlp.pipeline

[('tagger', <spacy.pipeline.pipes.Tagger at 0x7efe0806dfd0>),
 ('parser', <spacy.pipeline.pipes.DependencyParser at 0x7efe07ec6708>),
 ('ner', <spacy.pipeline.pipes.EntityRecognizer at 0x7efe07ec6768>)]

4. BREAKING THE CONTENTS BY SPAN AND SENTENCE METHODS

In [ ]:
dress_span = doc[0:20]
print(dress_span)
print(type(dress_span))

I really wanted this to work. alas, it had a strange fit for me. the straps would
<class 'spacy.tokens.span.Span'>


In [ ]:
i=0
for i,sent in enumerate(doc.sents):
    i+=1
    print(sent)
    if i==10:
      break

I really wanted this to work.
alas, it had a strange fit for me.
the straps would not stay up, and it had a weird fit under the breast.
it worked standing up, but the minute i sat down it fell off my shoulders.
the fabric was beautiful!
and i loved that it had pockets.

I love cute summer dresses and this one, especially because it is made out of linen, is unique.
it is very well-made with a design that is quite flattering.
i am 5 foot 6 and a little curvy with a 38 c bust
and i got a size 10.


In [ ]:
doc[17].is_sent_start

True

5. CONVERTING THE TOKENS INTO PANDAS DATAFRAME WITH **POS** AND **LEMMA** FOR EACH WORD

In [ ]:
dress_frame = pd.DataFrame()
o=0
for token in doc:
    dress_frame.loc[o, 'lemma']= token.lemma_
    dress_frame.loc[o, 'pos']= token.pos_
    dress_frame.loc[o, 'text']= token.text
    dress_frame.loc[o, 'lemma'] = token.dep_
    o=o+1
dress_frame[0:10]

,lemma,pos,text
0,nsubj,PRON,I
1,advmod,ADV,really
2,ROOT,VERB,wanted
3,nsubj,DET,this
4,aux,PART,to
5,ccomp,VERB,work
6,punct,PUNCT,.
7,intj,INTJ,alas
8,punct,PUNCT,","
9,nsubj,PRON,it


6. GROUPING THE TOKENS WITH PARTS OF SPEECH(POS)

In [ ]:
group_dress = dress_frame.groupby(['pos']).agg(
    {
      'text':'count' 
    })
group_dress['text'].sort_values(ascending=False)[0:15]

pos
NOUN     11003
DET       8329
PUNCT     8311
VERB      7634
PRON      6510
ADJ       6399
ADP       5133
AUX       4879
ADV       4787
CCONJ     3339
PART      1692
SCONJ     1361
SPACE     1327
NUM       1101
PROPN      822
Name: text, dtype: int64

7. FILTERING THE DATA WITH ONLY **ADJECTIVE** AND COUNTING ITS FREQUENCY

In [ ]:
group_dress_adj = dress_frame[dress_frame['pos']=='ADJ'].groupby(['text']).agg(
    {
      'text':'count'      
    })

In [ ]:
group_dress_adj['text'].sort_values(ascending=False)[0:5]

text
great          257
flattering     185
perfect        176
comfortable    162
small          149
Name: text, dtype: int64

In [ ]:
len(doc.vocab)

4205

8. Exploring NER Label **MONEY**

In [ ]:
for ent in doc.ents:
    if ent.label_ == 'MONEY':
     print(ent.text+' - '+ent.label_+' - '+str(spacy.explain(ent.label_)))

102# - MONEY - Monetary values, including unit
135 - MONEY - Monetary values, including unit
two cents - MONEY - Monetary values, including unit
128# - MONEY - Monetary values, including unit
140# 34d - MONEY - Monetary values, including unit
120# - MONEY - Monetary values, including unit
140# 5'3 - MONEY - Monetary values, including unit
15 bucks - MONEY - Monetary values, including unit
158 - MONEY - Monetary values, including unit
120# - MONEY - Monetary values, including unit
over $50 - MONEY - Monetary values, including unit
5'3 - MONEY - Monetary values, including unit
110# - MONEY - Monetary values, including unit
168 - MONEY - Monetary values, including unit
49 - MONEY - Monetary values, including unit
79 - MONEY - Monetary values, including unit
39;fuzzy&#39 - MONEY - Monetary values, including unit
over $250 - MONEY - Monetary values, including unit
#32b# - MONEY - Monetary values, including unit
120# max - MONEY - Monetary values, including unit
138 - MONEY - Monetary values

9. DOCUMNET PROPERTIES

In [ ]:
#Similar to Doc.ents, Doc.noun_chunks are another object property.
#Noun chunks are "base noun phrases" – flat phrases that have a noun as their head. 
#You can think of noun chunks as a noun plus the words describing the noun – for example, 
#in Sheb Wooley's 1958 song, a *"one-eyed, one-horned, flying, purple people-eater"* 
#would be one long noun chunk.

#https://spacy.io/usage/visualizers

In [ ]:
i=0
for chunk in doc.noun_chunks:
    i+=1
    print(chunk.text)
    if i ==15:
      break

I
it
a strange fit
me
the straps
it
a weird fit
the breast
it
i
it
my shoulders
the fabric
i
it


10. EXPLORING **DIFFERENT POS** FOR THE SAME WORD **SIZE**

In [ ]:
#Text: The original token text.
#Dep: The syntactic relation connecting child to head.
#Head text: The original text of the token head.
#Head POS: The part-of-speech tag of the token head.
#Children: The immediate syntactic dependents of the token.

i=0
for token in doc:
  if token.text =="size":
    if i == 15:
      break
    else:
     i+=1
     print(f'{token.text:{14}} {token.head.text:{12}} {token.head.pos_:{10}} {[child for child in token.children]}')

size           got          VERB       [a, 10]
size           to           ADP        []
size           down         ADP        [a]
size           xs           PROPN      []
size           xl           PROPN      []
size           to           ADP        []
size           was          AUX        [a, petite]
size           looked       VERB       [neither]
size           runs         VERB       [so, i, would, down, framed]
size           purchased    VERB       [the, 4, ,, fit]
size           small        ADJ        []
size           needed       VERB       [to, versital, ,, cute]
size           ordered      VERB       [my, regular, ,, medium]
size           to           PART       []
size           returned     VERB       [my]


In [ ]:
displacy.render(doc[0:500], style='ent', jupyter=True, options={'distance': 110})

#Stemming
not included in spacy and only used in nltk

Stemming on ADJECTIVE

In [ ]:
dress_adj = dress_frame[(dress_frame['pos']=='ADJ') & (dress_frame['lemma']=='acomp' )]
dress_adj[0:10]

,lemma,pos,text
54,acomp,ADJ,beautiful
84,acomp,ADJ,unique
98,acomp,ADJ,flattering
127,acomp,ADJ,difficult
181,acomp,ADJ,nice
200,acomp,ADJ,true
229,acomp,ADJ,lovely
253,acomp,ADJ,perfect
287,acomp,ADJ,adorable
292,acomp,ADJ,flattering


In [ ]:
list_of_adj_acomp_unique = [i for i in dress_adj['text'].unique()]
list_of_adj_acomp_unique[0:10]

['beautiful',
 'unique',
 'flattering',
 'difficult',
 'nice',
 'true',
 'lovely',
 'perfect',
 'adorable',
 'worse']

In [ ]:
p_stemmer = PorterStemmer()

In [ ]:
for word in list_of_adj_acomp_unique[0:10]:
    print(f'The stem of the word {word:{10}} is {p_stemmer.stem(word):{20}}')

The stem of the word beautiful  is beauti              
The stem of the word unique     is uniqu               
The stem of the word flattering is flatter             
The stem of the word difficult  is difficult           
The stem of the word nice       is nice                
The stem of the word true       is true                
The stem of the word lovely     is love                
The stem of the word perfect    is perfect             
The stem of the word adorable   is ador                
The stem of the word worse      is wors                


In [ ]:
# The Snowball Stemmer requires that you pass a language parameter
s_stemmer = SnowballStemmer(language='english')

In [ ]:
for word in list_of_adj_acomp_unique[0:10]:
    print(f'The stem of the word {word:{10}} is {s_stemmer.stem(word):{20}}')

The stem of the word beautiful  is beauti              
The stem of the word unique     is uniqu               
The stem of the word flattering is flatter             
The stem of the word difficult  is difficult           
The stem of the word nice       is nice                
The stem of the word true       is true                
The stem of the word lovely     is love                
The stem of the word perfect    is perfect             
The stem of the word adorable   is ador                
The stem of the word worse      is wors                


Stemming on VERB

In [ ]:
dress_verb = dress_frame[(dress_frame['pos']=='VERB') & (dress_frame['lemma']=='acomp' )]
dress_verb[0:10]

,lemma,pos,text
91,acomp,VERB,made
567,acomp,VERB,wearing
2526,acomp,VERB,made
4471,acomp,VERB,chested
4853,acomp,VERB,faded
6134,acomp,VERB,closed
8687,acomp,VERB,worried
9350,acomp,VERB,dressed
10675,acomp,VERB,looking
10781,acomp,VERB,endowed


In [ ]:
list_of_verb_acomp_unique = [i for i in dress_verb['text'].unique()]
list_of_verb_acomp_unique[0:10]

['made',
 'wearing',
 'chested',
 'faded',
 'closed',
 'worried',
 'dressed',
 'looking',
 'endowed',
 'pictured']

In [ ]:
for word in list_of_verb_acomp_unique[0:10]:
    print(f'The stem of the word {word:{10}} is {s_stemmer.stem(word):{20}}')

The stem of the word made       is made                
The stem of the word wearing    is wear                
The stem of the word chested    is chest               
The stem of the word faded      is fade                
The stem of the word closed     is close               
The stem of the word worried    is worri               
The stem of the word dressed    is dress               
The stem of the word looking    is look                
The stem of the word endowed    is endow               
The stem of the word pictured   is pictur              


# **Lemmatization**


1. In contrast to stemming, lemmatization looks beyond word reduction, and considers a language's full vocabulary to apply a morphological analysis to words. 
2. The lemma of 'was' is 'be' and the lemma of 'mice' is 'mouse'. 
3. Further, the lemma of 'meeting' might be 'meet' or 'meeting' depending on its use in a sentence.



Lemmetization on ADJECTIVE's

In [ ]:
#list_of_adj_acomp_unique = nlp(list_of_adj_acomp_unique)
i=0
for token in doc:
    if token.pos_ == 'ADJ' and token.dep_ =='acomp':
      if i<10:
        i+=1
        print(f'{token.text:{10}} {token.lemma_:{10}} {token.pos_:{6}} {token.dep_:{12}} {spacy.explain(token.tag_)}')
      else:
        break

beautiful  beautiful  ADJ    acomp        adjective
unique     unique     ADJ    acomp        adjective
flattering flattering ADJ    acomp        adjective
difficult  difficult  ADJ    acomp        adjective
nice       nice       ADJ    acomp        adjective
true       true       ADJ    acomp        adjective
lovely     lovely     ADJ    acomp        adjective
perfect    perfect    ADJ    acomp        adjective
adorable   adorable   ADJ    acomp        adjective
flattering flattering ADJ    acomp        adjective


Lemmetization on VERB's

In [ ]:
i=0
for token in doc:
    if token.pos_ == 'VERB' and token.dep_ =='acomp':
      if i<10:
        i+=1
        print(f'{token.text:{10}} {token.lemma_:{10}} {token.pos_:{6}} {token.dep_:{12}} {spacy.explain(token.tag_)}')
      else:
        break

made       make       VERB   acomp        verb, past participle
wearing    wear       VERB   acomp        verb, gerund or present participle
made       make       VERB   acomp        verb, past participle
chested    cheste     VERB   acomp        verb, past participle
faded      fade       VERB   acomp        verb, past participle
closed     close      VERB   acomp        verb, past participle
worried    worry      VERB   acomp        verb, past participle
dressed    dress      VERB   acomp        verb, past participle
looking    look       VERB   acomp        verb, gerund or present participle
endowed    endow      VERB   acomp        verb, past participle


# **Stop Words**

List of Default Stop Words

In [ ]:
stop_words = nlp.Defaults.stop_words
stop_word = [i for i in stop_words]
stop_word[0:10]

['around',
 'put',
 'very',
 'twelve',
 'per',
 'nor',
 'we',
 'more',
 'namely',
 'also']

Check if the word is a STOPWORD or not

In [ ]:
nlp.vocab['is']

In [ ]:
nlp.vocab['is'].is_stop

True

In [ ]:
nlp.vocab['mystery'].is_stop

False

Adding a stop word to the list of default list of STOPWORDS

In [ ]:
#Adding a stop word
#Add the word to the set of stop words. Use lowercase!
nlp.Defaults.stop_words.add('btw')

# Set the stop_word tag on the lexeme
nlp.vocab['btw'].is_stop = True

Removing a stop word from the list of default list of STOPWORDS

In [ ]:
# Remove the word from the set of stop words
nlp.Defaults.stop_words.remove('however')

# Remove the stop_word tag from the lexeme
nlp.vocab['however'].is_stop = False

In [ ]:
nlp.vocab['however'].is_stop

False

#Vocublary and Matching

Matcher

In [ ]:

#Here matcher is an object that pairs to the current 
#Vocab object. We can add and remove specific named matchers to matcher as needed.

#https://spacy.io/usage/rule-based-matching

#pattern1 = [{'LOWER': 'solarpower'}]
#pattern2 = [{'LOWER': 'solar'}, {'LOWER': 'power'}]
#pattern3 = [{'LOWER': 'solar'}, {'IS_PUNCT': True}, {'LOWER': 'power'}]

#matcher.add('SolarPower', None, pattern1, pattern2, pattern3)
#matcher.remove('SolarPower')


matcher = Matcher(nlp.vocab)

In [ ]:
pattern1 = [{'LOWER': 'tight','OP':'+'}]
pattern2 = [{'LOWER': 'petites'}]
matcher.add('sizes', None, pattern1,pattern2)
matcher

In [ ]:
found_matches = matcher(doc)
print(found_matches)

[(2738833442157585811, 1392, 1393), (2738833442157585811, 2658, 2659), (2738833442157585811, 3698, 3699), (2738833442157585811, 4138, 4139), (2738833442157585811, 5541, 5542), (2738833442157585811, 7383, 7384), (2738833442157585811, 7795, 7796), (2738833442157585811, 7884, 7885), (2738833442157585811, 8178, 8179), (2738833442157585811, 8437, 8438), (2738833442157585811, 8525, 8526), (2738833442157585811, 10724, 10725), (2738833442157585811, 12160, 12161), (2738833442157585811, 12831, 12832), (2738833442157585811, 13128, 13129), (2738833442157585811, 14542, 14543), (2738833442157585811, 15100, 15101), (2738833442157585811, 15812, 15813), (2738833442157585811, 15880, 15881), (2738833442157585811, 16810, 16811), (2738833442157585811, 18412, 18413), (2738833442157585811, 18733, 18734), (2738833442157585811, 18798, 18799), (2738833442157585811, 19671, 19672), (2738833442157585811, 21287, 21288), (2738833442157585811, 24605, 24606), (2738833442157585811, 25599, 25600), (2738833442157585811, 

In [ ]:
for match, starting, ending in found_matches[0:10]:
   for sent in doc.sents: 
    if sent.start< starting and sent.end > ending:
        print(doc[sent.start:sent.end])
    else:
        continue

i found the fit to be flattering -- fitted enough but not too loose or tight.
i do think the cut is on the trim side, but it isn't tight or fitted.
it is a bit looser on top (i'm 32c) and more form-fitting around the hips but not tight or clingy.
the end of the sleeves (where the buttons are) are very tight but
the arm holes were tight, but have very cute buttons if you look closely at the picture.  
both fit well, but the sleeves in the large were quite tight.
when it arrived and i tried it on, it fit great on my arms, wasn't too tight on my neck, but once it went down over my chest (36c), the dress never came back "in" to show my feminine waist/shape.
it is a perfect length and drapes well...not too tight at all.
ok, the arms were a little tight.
i also love how versatile it is--you can wear it as a dress as shown on the models, or as a tunic over tight jeans or jeggings/leggings.


PhraseMatcher

In [ ]:
phrase_matcher = PhraseMatcher(nlp.vocab)

In [ ]:
phrase_matcher = PhraseMatcher(nlp.vocab)
phrase = ['tight', 'extremely tight', 'too tight', 'wrong size']
phrase_patterns =  [nlp(text) for text in phrase]
print(phrase_patterns)
phrase_matcher.add('rsk', None, *phrase_patterns)

[tight, extremely tight, too tight, wrong size]


In [ ]:
# Build a list of matches:
phrase_matches = phrase_matcher(doc)
phrase_matches[0:10]

[(3674190656160917763, 1392, 1393),
 (3674190656160917763, 2658, 2659),
 (3674190656160917763, 3698, 3699),
 (3674190656160917763, 4138, 4139),
 (3674190656160917763, 5541, 5542),
 (3674190656160917763, 7383, 7384),
 (3674190656160917763, 7794, 7796),
 (3674190656160917763, 7795, 7796),
 (3674190656160917763, 7883, 7885),
 (3674190656160917763, 7884, 7885)]

In [ ]:
for match, starting, ending in phrase_matches[30:40]:
   for sent in doc.sents: 
    if sent.start< starting and sent.end > ending:
        print(doc[sent.start:sent.end])
    else:
        continue

the armholes fit perfectly though, if i had sized down they may have been too tight.
the slip was also slightly tight over hips.
i may have to try, though, as otherwise it's a great workable dress for summer--not too tight or revealing in the b

i may have to try, though, as otherwise it's a great workable dress for summer--not too tight or revealing in the b

for myself, the sleeves are tight and  so is the fit across the back and shoulders.  
one comment mentioned that the slip underneath was tight in the hips, yet as a size 30 in pants the slip was still fine.
This dress is beautiful but the bottom half was too tight for my shape.

This dress is beautiful but the bottom half was too tight for my shape.

the shoulders were only tight due to the lack of flexibility in the bottom - no elastic at the waist and no placket below the waist to increase the ease of mobility taking the garment on and off - so if i had to use the ladies' room, i had to plan ahead. 


the over-dress is beautifu

#Parts of Speech and NER

In [ ]:
POS_count = doc.count_by(spacy.attrs.POS)
for k,v in sorted(POS_count.items()):
  print(f'{k}, {doc.vocab[k].text} {v} ')

84, ADJ 6399 
85, ADP 5133 
86, ADV 4787 
87, AUX 4879 
89, CCONJ 3339 
90, DET 8329 
91, INTJ 87 
92, NOUN 11003 
93, NUM 1101 
94, PART 1692 
95, PRON 6510 
96, PROPN 822 
97, PUNCT 8311 
98, SCONJ 1361 
99, SYM 140 
100, VERB 7634 
101, X 37 
103, SPACE 1327 


In [ ]:
i=0
for token in doc:
    if token.pos_ == 'INTJ':
     if i<10:
      i+=1
      print(f'{token.text:{10}} {token.lemma_:{10}} {token.pos_:{6}} {token.tag_:{12}} {spacy.explain(token.tag_)}') 
     else:
       break

alas       alas       INTJ   UH           interjection
well       well       INTJ   UH           interjection
eh         eh         INTJ   UH           interjection
ha         ha         INTJ   UH           interjection
yeah       yeah       INTJ   UH           interjection
like       like       INTJ   UH           interjection
well       well       INTJ   UH           interjection
well       well       INTJ   UH           interjection
well       well       INTJ   UH           interjection
aw         aw         INTJ   UH           interjection


In [ ]:
spans = list(doc.sents)[0]
spans

I really wanted this to work.

# Named Entity Recognition

In [ ]:
displacy.render(doc[0:200], style='ent', jupyter=True, options={'distance': 110})

In [ ]:
def show_ents(doc):
    if doc.ents:
        for ent in doc.ents:
          if ent.label_ == 'PRODUCT':
            print(ent.text+' - '+ent.label_+' - '+str(spacy.explain(ent.label_)))
    else:
        print('No named entities found.')

document = show_ents(doc)
document

the s fit - PRODUCT - Objects, vehicles, foods, etc. (not services)
the s fit great - PRODUCT - Objects, vehicles, foods, etc. (not services)
p6 - PRODUCT - Objects, vehicles, foods, etc. (not services)
s - PRODUCT - Objects, vehicles, foods, etc. (not services)
a34b - PRODUCT - Objects, vehicles, foods, etc. (not services)
small/ - PRODUCT - Objects, vehicles, foods, etc. (not services)


In [ ]:
PRODUCT = doc.vocab.strings[u"PRODUCT"]
PRODUCT

386

# **Adding the custome entity tags**

In [ ]:
for i,token in enumerate(doc):
  if token.text == 'dress':
     print(token.text)
     print(i)
     break

dress
140


In [ ]:
doc[140]

dress

In [ ]:
list(doc)[130:160]

[up,
 because,
 the,
 material,
 has,
 no,
 give,
 .,
 the,
 perfect,
 dress,
 to,
 wear,
 to,
 italy,
 or,
 france,
 !,
 now,
 i,
 just,
 have,
 to,
 book,
 my,
 tickets,
 !,
 ,
 This,
 is]

In [ ]:
new_ent = Span(doc,140, 141, label=PRODUCT)
doc.ents = list(doc.ents)+ [new_ent]

In [ ]:
show_ents(doc)

dress - PRODUCT - Objects, vehicles, foods, etc. (not services)
the s fit - PRODUCT - Objects, vehicles, foods, etc. (not services)
the s fit great - PRODUCT - Objects, vehicles, foods, etc. (not services)
p6 - PRODUCT - Objects, vehicles, foods, etc. (not services)
s - PRODUCT - Objects, vehicles, foods, etc. (not services)
a34b - PRODUCT - Objects, vehicles, foods, etc. (not services)
small/ - PRODUCT - Objects, vehicles, foods, etc. (not services)


In [ ]:
phrase_matcher = PhraseMatcher(nlp.vocab)
phrase = ['dress']
phrase_patterns =  [nlp(text) for text in phrase]
print(phrase_patterns)
phrase_matcher.add('rsk', None, *phrase_patterns)

[dress]


In [ ]:
phrase_matches = phrase_matcher(doc)
phrase_matches[0:10]

[(3674190656160917763, 140, 141),
 (3674190656160917763, 163, 164),
 (3674190656160917763, 278, 279),
 (3674190656160917763, 362, 363),
 (3674190656160917763, 379, 380),
 (3674190656160917763, 420, 421),
 (3674190656160917763, 423, 424),
 (3674190656160917763, 467, 468),
 (3674190656160917763, 495, 496),
 (3674190656160917763, 510, 511)]

In [ ]:
new_ents_dress = [Span(doc, match[1], match[2], label=PRODUCT) for match in phrase_matches if match[1] != 140]
new_ents_dress[0:5]

[dress, dress, dress, dress, dress]

#Sentence Segmentation

In [ ]:
ss_list = list(doc.sents)[0:3]
ss_list

[I really wanted this to work.,
 alas, it had a strange fit for me.,
 the straps would not stay up, and it had a weird fit under the breast.]

Adding segmentation rule

In [ ]:
ss= [str(sent) for sent in ss_list]
ss= ".".join(ss)
print(type(ss))


<class 'str'>


In [ ]:
dec_ss= nlp(ss)
type(dec_ss)

spacy.tokens.doc.Doc

In [ ]:
for sent in dec_ss.sents:
  print(sent)

I really wanted this to work..
alas, it had a strange fit for me..
the straps would not stay up, and it had a weird fit under the breast.


In [ ]:
def set_custome_segmentations(doc):
  for token in doc[:-1]:
    if token.text == ",":
      doc[token.i+1].is_sent_start = True
  return doc

In [ ]:
#nlp.remove_i(set_custome_segmentation_rules, before='parser')
#nlp.remove_pipe("set_custome_segmentation")
nlp.pipe_names
nlp.add_pipe(set_custome_segmentations, before='parser')
print(nlp.pipe_names)

['tagger', 'set_custome_segmentations', 'parser', 'ner']


**BEFORE SEGMENTATION RULE**

In [ ]:
for sent in dec_ss.sents:
  print(sent)

I really wanted this to work..
alas, it had a strange fit for me..
the straps would not stay up, and it had a weird fit under the breast.


**AFTER SEGMENTAION RULE**

In [ ]:
doc_post_ss = nlp(ss)
for sent in doc_post_ss.sents:
  print(sent)

I really wanted this to work..
alas,
it had a strange fit for me..
the straps would not stay up,
and it had a weird fit under the breast.


In [ ]:
for sent in dec_ss.sents:
  print(sent)

I really wanted this to work..
alas, it had a strange fit for me..
the straps would not stay up, and it had a weird fit under the breast.
